In [17]:
path = "C:/bigcon/"

import pandas as pd
import numpy as np

In [27]:
cheoin = pd.read_excel(path + "input/additional_dataset/3. 산업중분류, 종사자 규모 및 읍면동별 사업체수, 종사자수.xls", sheet_name="처인", header=3,nrows = 2 )
gihung = pd.read_excel(path + "input/additional_dataset/3. 산업중분류, 종사자 규모 및 읍면동별 사업체수, 종사자수.xls", sheet_name="기흥", header=3,nrows = 2)
suji = pd.read_excel(path + "input/additional_dataset/3. 산업중분류, 종사자 규모 및 읍면동별 사업체수, 종사자수.xls", sheet_name="수지", header=3,nrows = 2)






In [28]:
def slicein(df):
    lst = np.array(df.iloc[0]) == '사업체수 Estab.'
    tmp = df.loc[1:,lst]
    return tmp

cheoin = slicein(cheoin)
gihung = slicein(gihung)
suji = slicein(suji)


In [29]:
# 종사자수 전처리
cheoin = cheoin.drop(['용인시', '처인구'], axis=1)
cheoin = cheoin.T

gihung = gihung.drop(['용인시', '기흥구'], axis=1)
gihung = gihung.T

suji = suji.drop(['용인시', '수지구'], axis=1)
suji = suji.T

works = pd.concat([cheoin, gihung, suji], axis=0)
works.columns = ['종사자수']

In [32]:
# 전기차

electric_car = pd.read_csv(path + 'input/additional_dataset/용인시 전기차 현황.csv', encoding='euc-kr')
electric_car['등록대수'] = electric_car['승용차수'] + electric_car['승합차수'] + electric_car['화물차수'] + electric_car['특수차수']

electric_car = electric_car.drop(['등록연도','연료별','용도별','승용차수', '승합차수', '화물차수', '특수차수'], axis=1)

cars = electric_car.groupby('시군구명').sum()

In [47]:
# 인구수
import re
pop = pd.read_csv(path + "input/additional_dataset/population.csv",encoding = 'CP949',header = 1)
popidx = pop.pop('연령별')
popidx[3] = '남사면'
pop.index = list(map(lambda x: re.sub(' ','',x),popidx))


pop_new = pop.copy()
pop_new['인구수'] = pop_new['20대'] + pop_new['30대'] + pop_new['40대'] + pop_new['50대'] + pop_new['60대']
pop_new = pop_new.drop(['20대', '30대', '40대', '50대', '60대','합계'], axis=1)



In [48]:
pop_cheoin = pop_new.iloc[:13,:]
pop_gihung = pop_new.iloc[13:28,:]
pop_suji = pop_new.iloc[28:,:]

In [49]:
pop_cheoin['인구비율'] = pop_cheoin['인구수'] / 188979
pop_gihung['인구비율'] = pop_gihung['인구수'] / 308373
pop_suji['인구비율'] = pop_suji['인구수'] / 261194

pop_cheoin['동별전기차'] = pop_cheoin['인구비율'] * 1826
pop_gihung['동별전기차'] = pop_gihung['인구비율'] * 2946
pop_suji['동별전기차'] = pop_suji['인구비율'] * 1963

C:\Users\ghrbs\AppData\Local\Temp/ipykernel_12064/3989078820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_cheoin['인구비율'] = pop_cheoin['인구수'] / 188979
C:\Users\ghrbs\AppData\Local\Temp/ipykernel_12064/3989078820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_gihung['인구비율'] = pop_gihung['인구수'] / 308373
C:\Users\ghrbs\AppData\Local\Temp/ipykernel_12064/3989078820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [50]:
pop_total = pd.concat([pop_cheoin, pop_gihung, pop_suji], axis=0)
pop_total.drop(['인구수','인구비율'], axis=1, inplace=True)

In [51]:
df = pd.merge(works, pop_total, left_index=True, right_index=True, how='left')
# NaN 나타나는지 확인 위해 일부러 left join 사용

In [52]:
df

종사자수       동별전기차
포곡읍    3795  237.300092
모현읍    4623  189.896115
이동읍    4111  141.719143
남사면    2578  158.010033
원삼면    1813   56.660603
백암면    2150   58.786341
양지면    3100  137.979776
중앙동    3938  182.378730
역삼동    3434  316.628800
유림동    2381  251.600516
동부동    2452   95.039851
구갈동    5320  302.784368
기흥동    2490  148.363767
서농동    1853  185.125118
구성동    2590  252.122871
마북동    1776  213.813875
보정동    3630  233.551128
상하동    1515  170.919263
신갈동    2842  270.413305
영덕1동   3572  211.771901
영덕2동   1902  123.677871
상갈동    1081   93.766283
보라동    2595  219.670088
동백1동   1466  208.158282
동백2동   4603  174.176961
동백3동   1009  143.128523
풍덕천1동  3223  255.963324
풍덕천2동  1890  270.245605
신봉동    1824  189.555602
죽전1동   3418  299.063757
죽전2동   1794  102.834403
동천동    3999  261.922223
상현1동   3006  252.798422
상현2동   1596  170.451235
성복동    2294  272.413842

In [53]:
def get_traffic(data):
    data['전기차_교통량'] = data['종사자수'] / data['종사자수'].sum() * data['동별전기차']
    return data

get_traffic(df)

종사자수       동별전기차    전기차_교통량
포곡읍    3795  237.300092   9.413816
모현읍    4623  189.896115     9.1769
이동읍    4111  141.719143   6.090206
남사면    2578  158.010033   4.258176
원삼면    1813   56.660603   1.073829
백암면    2150   58.786341   1.321207
양지면    3100  137.979776   4.471293
중앙동    3938  182.378730   7.507683
역삼동    3434  316.628800  11.365975
유림동    2381  251.600516     6.2622
동부동    2452   95.039851   2.436028
구갈동    5320  302.784368   16.83841
기흥동    2490  148.363767   3.861742
서농동    1853  185.125118   3.585888
구성동    2590  252.122871   6.826027
마북동    1776  213.813875   3.969491
보정동    3630  233.551128   8.862262
상하동    1515  170.919263   2.706822
신갈동    2842  270.413305   8.033562
영덕1동   3572  211.771901   7.907438
영덕2동   1902  123.677871      2.459
상갈동    1081   93.766283   1.059567
보라동    2595  219.670088   5.958875
동백1동   1466  208.158282   3.189948
동백2동   4603  174.176961   8.380843
동백3동   1009  143.128523    1.50964
풍덕천1동  3223  255.963324   8.623708
풍덕천2동  1890  270.245605   5.339203
신봉동    1824  189.555602   3.614244
죽전1동   3418  299.063757  10.685426
죽전2동   1794  102.834403   1.928488
동천동    3999  261.922223  10.949134
상현1동   3006  252.798422   7.943636
상현2동   1596  170.451235   2.843734
성복동    2294  272.413842   6.532488

In [55]:
df.to_excel(path + '/output/data/전기차 교통량.xlsx')